<a href="https://colab.research.google.com/github/DonghaeSuh/PyTorch_Basic/blob/main/MultiClass_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### File direction

In [ ]:
cd drive/MyDrive/pytorch

/content/drive/MyDrive/pytorch


### Library

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset,DataLoader

### CustomDatset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.a = df.iloc[:, 0].values
        self.b = df.iloc[:, 1].values
        self.c = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.y = list(map(self.string_to_vector, self.y))
        self.length = len(df)

    def string_to_vector(self, value):
        data = {"acute triangle": 0, "right triangle": 1, "obtuse triangle": 2}
        return data.get(value, None)
        # dictionary 자료형 key를 통해 value 불러오기 2가지 방법 
        # 1. data[원하는 key] => data 내에 원하는 key가 없을 경우 오류
        # 2. data.get(원하는 키) => data 내에 원한느 key가 없을 경우 None 반환

    def __getitem__(self, index):
        x = torch.FloatTensor(sorted([self.a[index], self.b[index], self.c[index]]))
        y = torch.LongTensor(self.y)[index]
        return x, y

    def __len__(self):
        return self.length

### CustomModel

In [ ]:
class CustomModel(nn.Module):
  def __init__(self):
    super(CustomModel,self).__init__()
    self.layer=nn.Sequential(
        nn.Linear(3,3)
    )

  def forward(self,x):
    x=self.layer(x)
    return x

### Data Load

In [ ]:
train_dataset = CustomDataset('./dataset.csv')
train_dataloader = DataLoader(train_dataset,batch_size=128,shuffle=True,drop_last=True)

### Compile

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
model=CustomModel().to(device)
criterion=nn.CrossEntropyLoss().to(device)  #Sotmax와 One-hot Encodingㅇ르 자체적을 수행
optimizer=optim.SGD(model.parameters(),lr=0.001)

### Train

In [ ]:
for epoch in range(10000):
  cost=0.0

  for x,y in train_dataloader:
    x= x.to(device)
    y= y.to(device)

    output=model(x)
    loss=criterion(output,y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost+=loss
  
  cost=cost/len(train_dataloader)

  if (epoch+1)%1000 ==0:
    print(f"Epoch : {epoch+1}, Cost : {cost:.3f}")

Epoch : 1000, Cost : 0.219
Epoch : 2000, Cost : 0.139


KeyboardInterrupt: ignored

### Evaluation

In [ ]:
for torch.no_grad():
  model.eval()
  classes={"acute triangle": 0, "right triangle": 1, "obtuse triangle": 2}
  
  inputs = torch.FloatTensor([
        [9.02, 9.77, 9.96], # 0 | acute triangle
        [8.01, 8.08, 8.32], # 0 | acute triangle
        [3.55, 5.15, 6.26], # 1 | right triangle
        [3.32, 3.93, 5.14], # 1 | right triangle
        [4.39, 5.56, 9.99], # 2 | obtuse triangle
        [3.01, 3.08, 9.98], # 2 | obtuse triangle
        [5.21, 5.38, 5.39], # 0 | acute triangle
        [3.85, 6.23, 7.32], # 1 | right triangle
        [4.16, 4.98, 8.54], # 2 | obtuse triangle
    ]).to(device)

    output=model(intputs)

    print("---------------")
    print(output)
    print(torch.round(F.softmax(output,dim=1),decimals=2))
    print(outputs.argmax(1))
    print(list(map(classes.get, outputs.argmax(1).tolist())))